In [1]:
import os
from pathlib import Path
import pandas as pd
import datetime
from IPython.display import display

pd.set_option('display.max_rows', 50)


In [2]:
data_path = "./data"
agg_path = "aggregated_data"
Path(f"{data_path}/{agg_path}").mkdir(parents=True, exist_ok=True)

languages = ["All", "English", "French", "German", "Italian"]
years = [str(2016 + i) for i in range(6)]
months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october",
         "november", "december"]
features = ["viewminutes", "streamedminutes", "maxchannels", "uniquechannels", "avgchannels", "maxviewers",
           "avgviewers", "avgratio"]

In [3]:
nr_Bubbles = 50
lang_dic = {}
datetime_list = {lang:[] for lang in languages}
for language in languages:
    Path(f"{data_path}/{agg_path}/{language}").mkdir(parents=True, exist_ok=True)
    language_files = sorted(os.listdir(data_path + "/" + language))
    lang_dic[language] = [None for _ in range(len(features))]
    for y in years:   
        for m_indx, m in enumerate(months):
            if (y!="2021") or (m_indx<3):
                try:
                    file_name = f"{language}-{y}{m}.csv"
                    data_df = pd.read_csv(f"{data_path}/{language}/{file_name}")[features + ["name"]]
                    data_df = data_df.set_index("name")
                    for indx, f in enumerate(features):
                        if (y==years[0]) and (m==months[0]):
                            lang_dic[language][indx] = data_df[[f]].sort_values(f, ascending=False).head(nr_Bubbles)
                        else:
                            lang_dic[language][indx] = lang_dic[language][indx].merge(data_df[[f]].sort_values(f, ascending=False).head(nr_Bubbles), left_index=True, right_index=True, how="outer")
                    datetime_list[language].append(datetime.date(int(y), m_indx+1, 1))
                except:
                    print("SOME ERROR:", language, y, m)

SOME ERROR: Italian 2017 october
SOME ERROR: Italian 2017 december
SOME ERROR: Italian 2018 february


In [4]:
lang_dic2 = {}
bust = False
for language in languages:
    lang_dic2[language] = [None for _ in range(len(features))]
    for y in years:   
        for m_indx, m in enumerate(months):
            if (y!="2021") or (m_indx<3):
                try:
                    file_name = f"{language}-{y}{m}.csv"
                    data_df = pd.read_csv(f"{data_path}/{language}/{file_name}")[features + ["name"]]
                    data_df = data_df.set_index("name")
                    dt = datetime.date(int(y), m_indx+1, 1)
                    for indx, f in enumerate(features):
                        indexes = lang_dic[language][indx].index
                        wanted = indexes
                        given = data_df.index
                        good_index = given.intersection(wanted)
                        if (y==years[0]) and (m==months[0]):
                            lang_dic2[language][indx] = data_df[[f]].loc[good_index]
                        else:
                            lang_dic2[language][indx] = lang_dic2[language][indx].merge(data_df[[f]].loc[good_index], left_index=True, right_index=True, how="outer")

                        lang_dic2[language][indx] = lang_dic2[language][indx].drop_duplicates()
                        lang_dic2[language][indx] = lang_dic2[language][indx][~lang_dic2[language][indx].index.duplicated(keep='first')]
                except:
                    print("SOME ERROR:", language, y, m)


SOME ERROR: Italian 2017 october
SOME ERROR: Italian 2017 december
SOME ERROR: Italian 2018 february


In [5]:
for language in languages:
    for indx in range(len(lang_dic2[language])):
        lang_dic2[language][indx].columns = datetime_list[language]
        lang_dic2[language][indx] = lang_dic2[language][indx].fillna(0)
        lang_dic2[language][indx] = lang_dic2[language][indx][~lang_dic2[language][indx].index.duplicated(keep='first')]


In [6]:
lang_dic2[language][indx].T

name,103,12HourChallenge,2Dark,30XX,360 No Scope Arena,60 Parsecs!,60 Seconds!,7th Sector,8 Ball Pool,9 Clues 2: The Ward,...,Zone of the Enders: The 2nd Runner,Zoo Tycoon,Zwift,ibb & obb,pop'n music FANTASIA,rFactor 2,skribbl.io,theHunter Classic,theHunter: Call of the Wild,Ōkami
2016-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,5.0
2016-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0
2016-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0
2016-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0
2016-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-01,0.0,0.0,0.0,0.0,0.0,0.0,177.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,6.56,11.94,4387.00,8.13,0.0
2020-12-01,0.0,0.0,0.0,0.0,0.0,0.0,39.00,0.0,26.00,0.0,...,11.0,0.0,18.0,10.50,0.0,7.14,12.17,306.88,38.38,18.0
2021-01-01,7.0,0.0,0.0,0.0,1504.0,0.0,26.17,0.0,1403.67,0.0,...,0.0,13.0,14.2,4.00,0.0,7.19,16.99,6.37,10.38,0.0
2021-02-01,0.0,0.0,0.0,754.5,0.0,0.0,10.50,0.0,0.00,0.0,...,0.0,0.0,12.0,37.00,0.0,6.66,14.59,8.11,26.74,10.0


In [14]:
for language in languages:
    for indx, f in enumerate(features):
        copy = lang_dic2[language][indx].T
        copy["date"] = copy.index
        copy.to_csv(f"{data_path}/{agg_path}/{language}/{language}_{f}_agg.csv")